In [1]:
# ML
from cProfile import label
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

import cv2
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import random_split

# Plots
import matplotlib.pyplot as plt

# Utils
import numpy as np
from collections import OrderedDict
from tqdm import tqdm, trange
import os
import sys
sys.path.insert(1, 'D:/Lucru/github-folder/itec-2022/')
from dataset import MyDataset
from network import MyModel

D:\Lucru\github-folder\itec-2022\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = MyDataset(samples_path="./dataset-aug/train/samples",
                          labels_path="./dataset-aug/train/labels.csv",
                          training=True)

In [3]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=1,
                              shuffle=True,
                              num_workers=0)

In [4]:
test_dataset = MyDataset(samples_path="./dataset-aug/test/samples",
                          labels_path="./dataset-aug/test/labels.csv")

In [5]:
test_dataloader = DataLoader(test_dataset,
                            batch_size=1,
                            shuffle=True,
                            num_workers=0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyModel().to(device)

In [7]:
print(model)

MyModel(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [8]:
# Unfreeze last layer
for param in model.resnet18.fc.parameters():
    param.requires_grad = True

In [9]:
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

batch_size = 1
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
epochs = 1
accs = []
losses = []
val_losses = []
val_accs = []
for epoch in (t:= trange(epochs)):
    model.train() # change resnet here
    # Get batches
    loss = 0.
    acc = 0.
    num_batches = 0
    for sample in trainloader:
        num_batches+=1
        source, label_shape, label_color, label_area = sample["image"], sample["label_shape"], sample["label_color"], sample["label_area"]
        
        source = source.cuda()
        label_shape = label_shape.cuda()
        label_color = label_color.cuda()
        label_area = label_area.cuda()
        
        output = model(source)
        predict_shape = output['shape']
        predict_color = output['color']
        predict_area = output['area']
        
        print(f'label shape: {label_shape.squeeze().shape}')
        print(f'predict shape: {predict_shape.shape}')
        
        #loss_shape = loss_function(torch.argmax(predict_shape, axis = 1), label_shape.squeeze().type(torch.float32))
        loss_shape = loss_function(predict_shape, label_shape)
        loss_color = loss_function(predict_color, label_color)
        loss_area = loss_function(predict_area, label_area)
        loss_batch = loss_shape + loss_color + loss_area
        loss += loss_batch.item()
        
        acc_shape = accuracy_score(torch.argmax(predict_shape.cpu(), axis = 1), label_shape.cpu())
        acc_color = accuracy_score(torch.argmax(predict_color.cpu(), axis = 1), label_color.cpu())
        acc_area = accuracy_score(torch.argmax(predict_area.cpu(), axis = 1), label_area.cpu())
        acc += (acc_shape + acc_color + acc_area) / 3
        
        optimizer.zero_grad() # don't forget this
        loss_batch.backward()
        optimizer.step()
    
    acc /= num_batches
    loss /= num_batches
    losses.append(loss)
    accs.append(acc)

    # Validation set
    resnet18.eval()
    num_batches = 0
    val_acc = 0.
    val_loss = 0.
    for sample in testloader:
        num_batches+=1
        source, label_shape, label_color, label_area = sample["image"], sample["label_shape"], sample["label_color"], sample["label_area"]
        
        output = model(source)
        predict_shape = output['shape']
        predict_color = output['color']
        predict_area = output['area']
        
        val_acc_shape = accuracy_score(torch.argmax(predict_shape.cpu(), axis = 1), label_shape.cpu())
        val_acc_color = accuracy_score(torch.argmax(predict_color.cpu(), axis = 1), label_color.cpu())
        val_acc_area = accuracy_score(torch.argmax(predict_area.cpu(), axis = 1), label_area.cpu())
        val_acc += (acc_shape + acc_color + acc_area) / 3
        
        loss_shape = loss_function(predict_shape, label_shape)
        loss_color = loss_function(predict_color, label_color)
        loss_area = loss_function(predict_area, label_area)
        loss_batch = loss_shape + loss_color + loss_area
        loss += loss_batch.item()   
        val_loss += loss_batch.item()
        
    val_acc /= num_batches
    val_loss /= num_batches
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    t.set_description(f"(Loss, Acc) -- Train : {round(loss, 2), round(acc, 2)}, Test : {round(val_loss, 2), round(val_acc, 2)}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

label shape: torch.Size([3])
predict shape: torch.Size([1, 1])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported